# NeuroFetal AI - Modular Training Pipeline

**Version 3.2** - Automated Secrets & TFLite Auto-Push

This notebook acts as a central launcher for the modular scripts in the codebase. It ensures consistency between local development and Cloud training.

### Steps:
1.  **Setup Environment**: Clone repo & install dependencies (Uses Colab Secrets).
2.  **Data Ingestion**: Process raw PhysioNet data.
3.  **Train**: Run the deep learning training pipeline.
4.  **Evaluate**: Run ensemble and uncertainty metrics.
5.  **Serve**: Launch the dashboard (optional).
6.  **Deploy**: Convert to TFLite and push to GitHub.

## 1. Setup Environment

In [ ]:
from google.colab import userdata
import os

# 1. GitHub Authentication
GITHUB_REPO = "Krishna200608/NeuroFetal-AI"

try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    print("✓ GitHub Token loaded from Secrets.")
except Exception as e:
    print("⚠️ Error loading GITHUB_TOKEN from Secrets. Falling back to manual input.")
    from getpass import getpass
    GITHUB_TOKEN = getpass("Enter GitHub Personal Access Token (PAT): ")

os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
os.environ['GITHUB_REPO'] = GITHUB_REPO

In [ ]:
# 2. Clone Repository
import shutil
import os

# CRITICAL FIX: Reset to /content before deleting the repo folder
# This prevents 'shell-init: error retrieving current directory'
try:
    os.chdir("/content")
except:
    pass

# Clean up any previous clone to avoid conflicts
if os.path.exists("/content/NeuroFetal-AI"):
    shutil.rmtree("/content/NeuroFetal-AI")

print("Cloning repository...")
!git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_REPO}.git

# Set paths
os.chdir("/content/NeuroFetal-AI")
print("Cloned successfully!")

In [ ]:
# 3. Install Dependencies
print("Installing libraries...")
!pip install -q wfdb shap scipy imbalanced-learn pyngrok filterpy scikit-learn matplotlib seaborn pandas numpy tensorflow streamlit plotly python-dotenv
print("Dependencies installed.")

## 2. Data Ingestion
Processes raw `.dat`/`.hea` files into clean `.npy` arrays for training.

In [ ]:
# Run the data ingestion script
!python Code/scripts/data_ingestion.py

## 2.5. Self-Supervised Pretraining (New)
Train the Masked Autoencoder (MAE) on the FHR data to learn robust features before supervision.
This saves encoder weights to `Code/models/pretrained_fhr_encoder.weights.h5`.

In [ ]:
# Run the SSL pretraining script
!python Code/scripts/pretrain.py

## 3. Training
Train the Tri-Modal Attention Fusion ResNet using 5-Fold Cross-Validation.
This script automatically handles:
*   Class Balancing (SMOTE)
*   Feature Extraction (CSP)
*   Model Checkpointing (saving best `.keras` files)

In [ ]:
# Run the main training script
!python Code/scripts/train.py

## 4. Advanced Evaluation
Generate metrics for:
1.  **Ensemble Performance**: Rank Averaging across folds (AUC maximization).
2.  **Uncertainty Quantification**: Monte Carlo Dropout confidence scores.

In [ ]:
print("Running Ensemble Evaluation (Rank Averaging)...")
!python Code/scripts/evaluate_ensemble.py

print("\nRunning Uncertainty Quantification...")
!python Code/scripts/evaluate_uncertainty.py

## 5. Launch Dashboard (Optional)
Run the Streamlit app directly from Colab using `ngrok`.
**Note**: You need an `NGROK_AUTH_TOKEN` set in your Colab Secrets.

In [ ]:
!pip install streamlit plotly python-dotenv pyngrok

In [ ]:
from google.colab import userdata

try:
    auth_token = userdata.get('NGROK_AUTH_TOKEN')
    print("✓ Ngrok Token loaded from Secrets.")
except Exception as e:
    print("⚠️ Error loading NGROK_AUTH_TOKEN from Secrets. Falling back to manual input.")
    from getpass import getpass
    auth_token = getpass("Enter Ngrok Auth Token manually: ")

if auth_token:
    with open("Code/.env", "w") as f:
        f.write(f"NGROK_AUTH_TOKEN={auth_token}\n")

print("Launching Streamlit App...")
!python Code/run_app.py

## 6. Convert to TFLite (Mobile Deployment)
Convert the best trained model to TFLite format and **PUSH to GitHub** automatically.
You don't need to download anything manually.

In [ ]:
print("Converting model to TFLite...")
!python Code/scripts/convert_to_tflite.py

print("\nPushing TFLite models to GitHub...")
# Configure Git identity (required for commit)
!git config --global user.email "krishnasikheriya001@gmail.com"
!git config --global user.name "Krishna200608"

# Add TFLite models
!git add Code/models/tflite/*.tflite

# Commit and Push
# We use '|| true' to prevent error if nothing to commit (e.g. running twice)
!git commit -m "chore: Auto-update TFLite models from Colab" || true
!git push origin main

print("✓ Models pushed to GitHub successfully! Check the repo.")